In [1]:
'''from fontTools.misc.classifyTools import Classifier
from sympy.tensor.tensor import tensor_mul

classification_nmaes={
    0: '上衣',
    1: '鞋子',
    2: '包',
    3: '下衣',
    4: '手表',
}'''

"from fontTools.misc.classifyTools import Classifier\nfrom sympy.tensor.tensor import tensor_mul\n\nclassification_nmaes={\n    0: '上衣',\n    1: '鞋子',\n    2: '包',\n    3: '下衣',\n    4: '手表',\n}"

In [2]:
'''import torch
from PIL import Image
import torchvision.transforms as T
from torch.utils.data import Dataset
import os
import pandas as pd

def sorted_alphanumeric(data):
    import re
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

#定义自定义数据集类FolderDataset
class FolderDataset(Dataset):
    def __init__(self,main_dir,transform=None):
        self.root_dir=main_dir
        self.transform=transform
        self.all_image=sorted_alphanumeric(os.listdir(main_dir))
        self.classification_nmaes=pd.read_csv('../common/fashion-labels.csv')
        self.laberl_dict=dict(zip(self.classification_nmaes['id'],self.classification_nmaes['target']))  #拉链
    def __len__(self):
        return len(self.all_image)
    def __getitem__(self,idx):
        img_loc=os.path.join(self.root_dir,self.all_image[idx])

        image=Image.open(img_loc).convert('RGB')
        img_flag_id=self.labell_dict[idx]

        if self.transform:
            image=self.transform(image)
        else:
            raise RuntimeError("Transform is not defined")
        return tensor_image,img_flag_id
'''

'import torch\nfrom PIL import Image\nimport torchvision.transforms as T\nfrom torch.utils.data import Dataset\nimport os\nimport pandas as pd\n\ndef sorted_alphanumeric(data):\n    import re\n    convert = lambda text: int(text) if text.isdigit() else text.lower()\n    alphanum_key = lambda key: [convert(c) for c in re.split(\'([0-9]+)\', key)]\n    return sorted(data, key=alphanum_key)\n\n#定义自定义数据集类FolderDataset\nclass FolderDataset(Dataset):\n    def __init__(self,main_dir,transform=None):\n        self.root_dir=main_dir\n        self.transform=transform\n        self.all_image=sorted_alphanumeric(os.listdir(main_dir))\n        self.classification_nmaes=pd.read_csv(\'../common/fashion-labels.csv\')\n        self.laberl_dict=dict(zip(self.classification_nmaes[\'id\'],self.classification_nmaes[\'target\']))  #拉链\n    def __len__(self):\n        return len(self.all_image)\n    def __getitem__(self,idx):\n        img_loc=os.path.join(self.root_dir,self.all_image[idx])\n\n        image=I

In [3]:
'''import re
import torch
import torchvision.transforms as T
#检查是否有可用的GPU,否则使用CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
transforms=T.Compose([
    T.Resize((64,64)),
    T.ToTensor(),

])
print('----------正在创建数据表----------')
full_dataset=FolderDataset('../common/dataset/', transform=transforms)
train_size=int(0.75*len(full_dataset))
val_size=len(full_dataset)-train_size
full_dataset,val_size=torch.utils.data.random_split(full_dataset,[train_size,val_size])


print('----------数据表创建完成----------')
print('----------正在创建数据加载器----------')

#定义批次大小
batch=32

#创建训练集的DataLoder,批次大小为32
train_loder=torch.utils.data.DataLoader(full_dataset,batch_size=batch,shuffle=True)
#创建验证集的DataLoder,批次大小为32
val_loader=torch.utils.data.DataLoader(val_size,batch_size=batch)

#创建完整集的DataLoder,批次大小为32
full_loder=torch.utils.data.DataLoader(full_dataset,batch_size=batch)

print('----------数据加载器创建完成----------')

#遍历训练集的DataLoder，打印每第一个批次的数据
for x,y in train_loder:
    print('图像张量形状:',x.shape)
    print('标签:',y.shape)
    break

'''









'import re\nimport torch\nimport torchvision.transforms as T\n#检查是否有可用的GPU,否则使用CPU\nif torch.cuda.is_available():\n    device = torch.device("cuda")\nelse:\n    device = torch.device("cpu")\ntransforms=T.Compose([\n    T.Resize((64,64)),\n    T.ToTensor(),\n\n])\nprint(\'----------正在创建数据表----------\')\nfull_dataset=FolderDataset(\'../common/dataset/\', transform=transforms)\ntrain_size=int(0.75*len(full_dataset))\nval_size=len(full_dataset)-train_size\nfull_dataset,val_size=torch.utils.data.random_split(full_dataset,[train_size,val_size])\n\n\nprint(\'----------数据表创建完成----------\')\nprint(\'----------正在创建数据加载器----------\')\n\n#定义批次大小\nbatch=32\n\n#创建训练集的DataLoder,批次大小为32\ntrain_loder=torch.utils.data.DataLoader(full_dataset,batch_size=batch,shuffle=True)\n#创建验证集的DataLoder,批次大小为32\nval_loader=torch.utils.data.DataLoader(val_size,batch_size=batch)\n\n#创建完整集的DataLoder,批次大小为32\nfull_loder=torch.utils.data.DataLoader(full_dataset,batch_size=batch)\n\nprint(\'----------数据加载器创建完成----------\')

In [4]:
'''import torch.nn as nn
import torch.nn.functional as F

class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1=nn.Linear(16*16*16,128)


    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(x)

        x=self.pool(F.relu(self.conv2(x)))
        x=self.pool(x)

        x=x.reshape(x.shape[0],-1)

        x=self.fc1(x)
        x=F.log_softmax(x,dim=1)
        return x



# 实例化CNN模型
model=ClassificationModel()

print(model)

#获取训练加载器的迭代器
train_iterator=iter(train_loder)
#获取下一个批次的数据
images,labels=next(train_iterator)
print('数据经过模型后向前传递后的维度:',model(images).shape)
'''

"import torch.nn as nn\nimport torch.nn.functional as F\n\nclass ClassificationModel(nn.Module):\n    def __init__(self):\n        super(ClassificationModel,self).__init__()\n        self.conv1=nn.Conv2d(in_channels=3,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))\n        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))\n        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))\n        self.fc1=nn.Linear(16*16*16,128)\n\n\n    def forward(self,x):\n        x=self.pool(F.relu(self.conv1(x)))\n        x=self.pool(x)\n\n        x=self.pool(F.relu(self.conv2(x)))\n        x=self.pool(x)\n\n        x=x.reshape(x.shape[0],-1)\n\n        x=self.fc1(x)\n        x=F.log_softmax(x,dim=1)\n        return x\n\n\n\n# 实例化CNN模型\nmodel=ClassificationModel()\n\nprint(model)\n\n#获取训练加载器的迭代器\ntrain_iterator=iter(train_loder)\n#获取下一个批次的数据\nimages,labels=next(train_iterator)\nprint('数据经过模型后向前传递后的维度:',model(images).shape)\n"

In [5]:
# 定义一个字典，将数字标签映射为对应的中文名称
classification_names = {
    0: '上身衣服',  # 数字 0 对应“上身衣服”
    1: '鞋',       # 数字 1 对应“鞋”
    2: '包',       # 数字 2 对应“包”
    3: '下身衣服',  # 数字 3 对应“下身衣服”
    4: '手表'      # 数字 4 对应“手表”
}

In [6]:
import torch  # 导入 PyTorch 库，用于深度学习任务
from PIL import Image  # 导入 PIL 库中的 Image 模块，用于图像处理
import os  # 导入 os 库，用于文件和目录操作
from torch.utils.data import Dataset  # 导入 PyTorch 的 Dataset 类，用于创建自定义数据集
import torchvision.transforms as T  # 导入 torchvision 的 transforms 模块，用于图像预处理
import pandas as pd  # 导入 pandas 库，用于数据处理和分析

def sorted_alphanumeric(data):
    """按字母数字混合顺序对文件名进行排序（例如：img1, img2, ..., img10）"""
    # 定义转换函数：将数字部分转换为整数，非数字部分转换为小写
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    # 生成排序键：用正则分割字符串，分别处理数字和非数字部分
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    # 按生成的键排序
    return sorted(data, key=alphanum_key)

# 定义自定义数据集类 FolderDataset，继承自 PyTorch 的 Dataset 类
class FolderDataset(Dataset):
    def __init__(self, main_dir, transform=None):
        """
        初始化方法。
        :param main_dir: 图像文件夹的主路径
        :param transform: 图像预处理操作（如缩放、裁剪、归一化等），默认为 None
        """
        self.main_dir = main_dir  # 存储主目录路径
        self.transform = transform  # 存储图像预处理操作
        self.all_imgs = sorted_alphanumeric(os.listdir(main_dir))  # 获取主目录下的所有图像文件名
        # 读取包含分类标签的 CSV 文件
        self.classifications = pd.read_csv('../common/fashion-labels.csv')  # 从指定路径加载 CSV 文件到 DataFrame
        # 将数据类型转换为字典，提升查询效率
        self.label_dict = dict(zip(self.classifications['id'], self.classifications['target']))

    def __len__(self):
        """
        返回数据集中图像的数量。
        """
        return len(self.all_imgs)  # 返回图像文件的总数

    def __getitem__(self, idx):
        """
        根据索引 idx 获取图像及其标签。
        :param idx: 图像的索引
        :return: 预处理后的图像张量和对应的标签
        """
        # print("index: ", idx)
        img_loc = os.path.join(self.main_dir, self.all_imgs[idx])  # 获取第 idx 张图像的完整路径
        image = Image.open(img_loc).convert("RGB")  # 打开图像并将其转换为 RGB 格式
        img_flag_id = self.label_dict[idx]  # 在分类数据中查找对应的标签

        # 对图像进行预处理（如果定义了 transform）
        if self.transform is not None:
            tensor_image = self.transform(image)  # 应用预处理操作，将图像转换为张量
        else:
            raise RuntimeError("transform参数不能为None，需指定预处理方法")

        return tensor_image, img_flag_id  # 返回图像张量和标签

In [7]:
import re
import torch
# 检查是否有可用的GPU，如果有则使用GPU，否则使用CPU
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# 定义数据预处理流程：将图像调整为64x64大小，并将其转换为Tensor
transforms = T.Compose([T.Resize((64, 64)), T.ToTensor()])

# 打印提示信息，表示正在创建数据集
print("------------ 正在创建数据集 ------------")
# 使用FolderDataset类加载数据集，数据集路径为'./image_similarity/dataset/'，并应用定义好的预处理流程
full_dataset = FolderDataset('../common/dataset/', transforms)

# 计算训练集和验证集的大小，训练集占数据集的75%，验证集占剩余的25%
train_size = int(0.75 * len(full_dataset))
val_size = len(full_dataset) - train_size

# 使用random_split函数将数据集随机划分为训练集和验证集
train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size]
)

# 打印提示信息，表示数据集已创建
print("------------ 数据集创建完成 ------------")

# 打印提示信息，表示正在创建DataLoader
print("------------ 创建数据加载器 ------------")

# 定义批次大小为32
batch = 32

# 创建训练集的DataLoader，批次大小为32，打乱数据顺序，并丢弃最后一个不完整的批次
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch, shuffle=True, drop_last=True
)

# 创建验证集的DataLoader，批次大小为32，不打乱数据顺序
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch
)

# 创建完整数据集的DataLoader，批次大小为32，不打乱数据顺序
full_loader = torch.utils.data.DataLoader(
    full_dataset, batch_size=batch
)

print("------------ 数据加载器创建完成 ------------")

# 遍历训练集的DataLoader，获取第一个批次的数据并打印其形状
for (x, y) in train_loader:
    print(x.shape)  # 打印输入数据的形状
    print(y.shape)  # 打印标签数据的形状
    break  # 获取第一个批次后退出循环

------------ 正在创建数据集 ------------
------------ 数据集创建完成 ------------
------------ 创建数据加载器 ------------
------------ 数据加载器创建完成 ------------
torch.Size([32, 3, 64, 64])
torch.Size([32])


In [8]:
import torch.nn as nn  # 导入 PyTorch 的神经网络模块，用于构建神经网络
import torch.nn.functional as F  # 导入 PyTorch 的函数模块，包含激活函数、损失函数等

'''class Classifier(nn.Module):
    def __init__(self, in_channels=1, num_classes=5):
        super(Classification, self).__init__()
        # 定义第一个卷积层，输入通道数为3，输出通道数为8，卷积核大小为3x3，步幅为1，填充为1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        # 使用填充为1的卷积操作，输出特征图的尺寸与输入相同（Same convolutions）

        # 定义最大池化层，池化核大小为2x2，步幅为2
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # 定义第二个卷积层，输入通道数为8，输出通道数为16，卷积核大小为3x3，步幅为1，填充为1
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        # 上一层的输出通道数为8，因此这一层的输入通道数为8

        # 定义全连接层，输入大小为16*16*16，输出大小为num_classes（分类数）
        self.fc1 = nn.Linear(16*16*16, num_classes)
        # 输入大小为16是因为conv2的输出通道数为16，16*16是因为经过两次池化后，特征图的尺寸为16x16

    def forward(self, x):
        # 第一层卷积 + ReLU激活函数
        x = F.relu(self.conv1(x))
        # print("经过第一层卷积和ReLU激活函数后的维度：", x.shape)
        # 池化操作
        x = self.pool(x)
        # print("经过第一层池化后的维度：", x.shape)
        # 第二层卷积 + ReLU激活函数
        x = F.relu(self.conv2(x))
        # print("经过第二层卷积和ReLU激活函数后的维度：", x.shape)
        # 池化操作
        x = self.pool(x)
        # print("经过第二层池化后的维度：", x.shape)
        # 将特征图展平为一维向量，保留batch_size维度，其余维度展平
        x = x.reshape(x.shape[0], -1)
        # print("经过一维展平后的维度：", x.shape)

        # 全连接层
        x = self.fc1(x)
        # print("经过全连接层的维度：", x.shape)

        # 经过log_softmax处理后返回，该函数通常用于多分类任务
        x = F.log_softmax(x, dim=1)
        # print("经过log_softmax处理后的维度：", x.shape)
        return x

# 实例化CNN模型
model = Classifier()
# 打印模型结构
print(model)
'''

class Classifier(nn.Module):
    def __init__(self, in_channels=1, num_classes=5):
        # 关键修正：把 Classification 改成 Classifier
        super(Classifier, self).__init__()

        # 修正：用参数 in_channels 统一输入通道数
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1)
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1)
        )
        self.fc1 = nn.Linear(16*16*16, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = F.log_softmax(x, dim=1)
        return x

# 实例化模型
model = Classifier()
# 打印模型结构
print(model)

# 获取训练数据加载器的迭代器
it = iter(train_loader)
# 获取下一个批次的数据
X_batch, y_batch = next(it)
# 打印模型前向传播输出的形状
print("数据经过模型向前传递后的维度：", model.forward(X_batch).shape)

Classifier(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=5, bias=True)
)


RuntimeError: Given groups=1, weight of size [8, 1, 3, 3], expected input[32, 3, 64, 64] to have 1 channels, but got 3 channels instead

In [ ]:
# 指定损失函数为交叉熵损失函数，适用于多分类任务
criterion = nn.CrossEntropyLoss()

# 指定优化器为Adam优化器，优化对象是模型的所有参数，学习率为0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 指定训练的轮数为20
n_epochs = 20

# 将模型移动到指定的设备（GPU或CPU）
model.to(device)

# 开始训练循环，遍历每个epoch
for epoch in range(1, n_epochs+1):
    # 初始化训练损失为0.0，用于累加每个批次的损失
    train_loss = 0.0

    ###################
    # 训练模型 #
    ###################
    for (x, y) in train_loader:  # 遍历训练集的每个批次
        # 将输入数据x和标签y移动到指定的设备（GPU或CPU）
        x = x.to(device)
        y = y.to(device)

        # 清除优化器中所有参数的梯度，避免梯度累积
        optimizer.zero_grad()
        # 前向传播：将输入数据x传递给模型，得到输出结果
        output = model(x)
        # 计算损失值，output是模型的预测结果，y是真实标签
        loss = criterion(output, y)
        # 反向传播：计算损失相对于模型参数的梯度
        loss.backward()
        # 更新模型参数：执行一次优化步骤
        optimizer.step()
        # 累加当前批次的损失值，乘以批次大小（x.size(0)）以得到总损失
        train_loss += loss.item() * x.size(0)
    # 计算平均训练损失，总损失除以训练集的批次数量
    train_loss = train_loss / len(train_loader)
    # 打印当前epoch的训练损失
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

In [1]:
# 导入matplotlib库用于绘图
import matplotlib.pyplot as plt
# 在Jupyter Notebook中内嵌显示matplotlib图像
%matplotlib inline
# 导入numpy库用于数值计算
import numpy as np

# 获取验证集的一个批次数据
dataiter = iter(val_loader)
images, labels = next(dataiter)

# 将模型移动到CPU上，因为后续操作需要在CPU上执行
model.to('cpu')

# 获取模型对输入图像的输出结果
output = model(images)
# 将输出结果从计算图中分离，并转换为numpy数组
output = output.detach().numpy()
# 打印输出结果的形状
print(output.shape)

# 将输入图像从Tensor转换为numpy数组
images = images.numpy()
# 调整图像的维度顺序，从(C, H, W)变为(H, W, C)，以便matplotlib正确显示
images = np.moveaxis(images, 1, -1)
# 打印调整后的图像形状
print(images.shape)

# 遍历每个图像及其对应的模型输出结果
for img, label in zip(images, output):
    # 显示当前图像
    plt.imshow(img)
    plt.show()
    print("label: ", label)
    print("label.shape: ", label.shape)
    # 打印模型预测的类别名称
    print(classification_names[np.argmax(label)])


ImportError: Matplotlib requires numpy>=1.23; you have 1.22.4